In [ ]:
# # TODO fix videos with ids: ...","'

# for id_ in G.nodes:
#     if '...","' in id_:
#         print(G.nodes[id_])

In [ ]:
import os
import sys

import networkx as nx
from neo4j import GraphDatabase

sys.path.append(os.path.abspath(".."))
import matplotlib.pyplot as plt

from yourtube.neo4j_queries import *
from yourtube.scraping import *

driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "yourtube"))

In [ ]:
with driver.session() as s:
    s.write_transaction(create_constraints)

In [ ]:
# id_ = "HxGELmaqSHU"
# id_ = 'YOrJJ...","'
id_ = "Fa_un5wXC-ch"

In [ ]:
# down_id = "3zUBBuqumbQ"

# content = get_content(down_id)
# scrape_content(content, down_id, driver)

In [ ]:
# G = nx.DiGraph()

# scrape_content(content, id_, G)

In [ ]:
def get_video(tx, id_):
    res = tx.run(
        """
        MATCH (v:video {video_id: $id_})
        RETURN v
        """,
        id_=id_,
    ).single()
    if res is not None:
        return res[0]
    else:
        return None

In [ ]:
#  G = load_graph()

In [ ]:
# delete all
with driver.session() as s:
    s.write_transaction(delete_all)

In [ ]:
with driver.session() as s:
    node_pairs = s.read_transaction(get_all_user_relevant_video_info, "default")

In [ ]:
G = nx.DiGraph()
for v1, v2 in node_pairs:
    G.add_node(v1["video_id"], **v1)
    G.add_node(v2["video_id"], **v2)
    G.add_edge(v1["video_id"], v2["video_id"])

In [ ]:
G.nodes["risSXbH-u9w"]

In [ ]:
with driver.session() as s:
    playlist_info = s.read_transaction(get_all_user_relevant_playlist_info, "default")

In [ ]:
playlist_info[0]

In [ ]:
for playlist_name, video_id, time_added in playlist_info:
    if video_id not in G.nodes:
        # this means the video had no recommended videos, and wasn't matched by the previous step
        # so it's probably down
        continue
    G.nodes[video_id]["from"] = playlist_name
    G.nodes[video_id]["time_added"] = time_added